In [1]:
from google.colab import drive
drive.mount('/content/drive')
OBJ1_OBJ2_JSON = '/content/drive/MyDrive/cs5242_project/jsons/object1_object2.json'
RELATIONSHIP_JSON = '/content/drive/MyDrive/cs5242_project/jsons/relationship.json'
CAPTIONS_FILE = '/content/drive/MyDrive/cs5242_project/jsons/training_annotation.json'
TRAIN_FEATS_DIR ='/content/drive/MyDrive/cs5242_project/extracted_feats/9_inceptionV4/train_feats/'
TEST_FEATS = '/content/drive/MyDrive/cs5242_project/extracted_feats/9_inceptionV4/test_feats/'

Mounted at /content/drive


In [2]:
import json
import os
import numpy as np
import torch
from torch.utils.data import Dataset,  DataLoader
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import pandas as pd

In [4]:
with open(OBJ1_OBJ2_JSON, 'r') as obj:
       obj1_obj2 = json.load(obj)
with open(RELATIONSHIP_JSON, 'r') as relat:
       relationship = json.load(relat)
with open(CAPTIONS_FILE, 'r') as anno:
    training_anno = json.load(anno)

#Preparing Vocabulary
relationship_vocab_stoi = relationship
relationship_vocab_itos = {}
for k,v in relationship_vocab_stoi.items():
    relationship_vocab_itos[v] = k

relationship_new = {}
idx = 35

for k,v in relationship.items():
    relationship_new[k] = idx
    idx= idx+1
    
vocab_stoi = {**obj1_obj2, **relationship_new}
vocab_stoi["<BOS>"] = 117


vocab_itos = {}
for k,v in vocab_stoi.items():
    vocab_itos[v] = k

training_anno_new ={}
for k,v in training_anno.items():
    training_anno_new[k] = training_anno[k]
    training_anno_new[k][1] = vocab_stoi[relationship_vocab_itos[training_anno_new[k][1]]]  

In [5]:
class MyDataset(Dataset):

    def __init__(self, features_dir, training_anno_new):
        super(MyDataset, self).__init__()
        
        self.features = []
        self.captions = []

        self.feat_dir = features_dir
                
        self.captions_file = training_anno_new
        
        for f in os.listdir(self.feat_dir):
            feats = np.load(os.path.join(self.feat_dir,f))
            self.features.append(feats)
            self.captions.append(self.captions_file[f.split('.')[0]])
        
    def __getitem__(self, idx):
        return torch.tensor(self.features[idx]), torch.tensor([117] + self.captions[idx])  
    
    def __len__(self):
        return len(self.features)

In [6]:
train_dataset = MyDataset(TRAIN_FEATS_DIR, training_anno_new)
trainloader = DataLoader(
        dataset=train_dataset,
        batch_size=1,
        shuffle=True,
    )

In [7]:
for feats, captions in trainloader:
    print(feats.shape)
    print(captions.shape)
    break

torch.Size([1, 30, 1536])
torch.Size([1, 4])


In [14]:
class Encoder(nn.Module):
    def __init__(self, embed_size=256, hidden_size=256, num_layers=1, stacked_dropout=0, bidirectional=False, rnn='LSTM'):
        super(Encoder, self).__init__()
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1536, embed_size)

        if rnn  == 'LSTM':
            self.rnn1 = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout = stacked_dropout)
        if rnn  == 'GRU':
            self.rnn1 = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout = stacked_dropout)

        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, features):
        
        features = self.fc(features) 
        
        features = self.relu(features)
        
        features = self.dropout(features)

        _ , hidden1 = self.rnn1(features)

        return _ , hidden1

In [15]:
class Decoder(nn.Module):
    def __init__(self, embed_size=256, hidden_size=256, num_layers=1, vocab_size=118, stacked_dropout=0, bidirectional=False, rnn='LSTM'):
        super(Decoder, self).__init__()
        
        self.dropout = nn.Dropout(p=0.5)
        self.embed = nn.Embedding(vocab_size, embed_size)

        if rnn  == 'LSTM':
            self.rnn2 = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout = stacked_dropout)
        if rnn  == 'GRU':
            self.rnn2 = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout = stacked_dropout)

        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, vocab_size)   
        else:
            self.fc = nn.Linear(hidden_size, vocab_size) 

    
    def forward(self, caption, hidden1):
        
        caption = caption.unsqueeze(dim=1)
        
        embedding = self.embed(caption)
        
        embedding = self.dropout(embedding)
                
        out2, hidden2 = self.rnn2(embedding, hidden1) 

        predictions = self.fc(out2) 
        
        predictions = predictions.squeeze(1)

        return predictions, hidden2

In [16]:
class EncoderDecoder(nn.Module):
    def __init__(self):
        super(EncoderDecoder, self).__init__()
        self.encoder = Encoder(num_layers=1, bidirectional=False, stacked_dropout=0, rnn='LSTM')
        self.decoder = Decoder(num_layers=1, bidirectional=False, stacked_dropout=0, rnn='LSTM')
    

    def forward(self, features, caption):   

        _ , hidden = self.encoder(features)

        outputs = torch.zeros(3, caption.shape[0], 118)
        
        x = caption[:, 0]

        for t in range(0, 3):

            pred, hidden = self.decoder(x, hidden)
            
            outputs[t] = pred

            _ , max_index = torch.max(pred, dim=1)

            x = max_index

        return outputs
    
    
    def caption_image(self, features, max_length=3):
        result_caption = []

        with torch.no_grad():

            _ , hidden1 = self.encoder(features.unsqueeze(0).to('cuda'))

            top5_array = torch.zeros(3, 5)

            for t in range(max_length):
                if t==0:
                    initial_input = torch.tensor([vocab_stoi['<BOS>']]).unsqueeze(dim=1).to('cuda')
                    initial_input_emb = self.decoder.embed(initial_input)
                    out2, hidden2 = self.decoder.rnn2(initial_input_emb, hidden1)
                else:
                    out2, hidden2 = self.decoder.rnn2(next_word_emd, hidden2)
                    
                output = self.decoder.fc(out2).squeeze(1)

                _, top5_idxs = torch.topk(output, 5, dim=1)
                top5_array[t] = top5_idxs

                _, max_index = torch.max(output, dim=1)
                result_caption.append(max_index.item())

                next_word_emd = self.decoder.embed(max_index).unsqueeze(1)

        return top5_array, [vocab_itos[idx] for idx in result_caption]

In [17]:
model_name = 'MyModel'
model_path = '/content/drive/MyDrive/cs5242_project/saved_models/' + model_name + '/'
if not os.path.isdir(model_path):
    os.mkdir(model_path)

In [18]:
def train():

    device = "cuda" 
    learning_rate = 3e-4
    num_epochs = 25

    model = EncoderDecoder().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)


    model.train()

    for epoch in range(num_epochs):
        print(f'Epoch: {epoch+1}/{num_epochs}')

        for idx, (features, captions) in tqdm(enumerate(trainloader), total=len(trainloader), leave=False):
            features = features.to(device)
            captions = captions.to(device)

            outputs = model(features, captions).to(device)

            optimizer.zero_grad()

            outputs = outputs.reshape(-1, outputs.shape[2])
            captions = captions[:,1:].reshape(-1)

            loss = criterion(outputs, captions)          
            loss.backward()
            optimizer.step()

    torch.save(model,  model_path + 'final.pth')

In [19]:
train()

  0%|          | 0/447 [00:00<?, ?it/s]

Epoch: 1/25


  4%|▍         | 18/447 [00:00<00:02, 179.52it/s]

Epoch: 2/25


  4%|▍         | 19/447 [00:00<00:02, 180.70it/s]

...

Epoch: 23/25


  4%|▍         | 18/447 [00:00<00:02, 173.54it/s]

Epoch: 24/25


  4%|▍         | 18/447 [00:00<00:02, 173.30it/s]

Epoch: 25/25


In [27]:
m = 'final.pth'
mod = torch.load(model_path+m)
mod.eval()
test_feat = torch.tensor(np.load(TEST_FEATS +'000118.npy'))
_, best = mod.caption_image(test_feat)
print(best)

['elephant', 'stand_right', 'elephant']


In [28]:
print(mod)

EncoderDecoder(
  (encoder): Encoder(
    (relu): ReLU()
    (fc): Linear(in_features=1536, out_features=256, bias=True)
    (rnn1): LSTM(256, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embed): Embedding(118, 256)
    (rnn2): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=118, bias=True)
  )
)


In [30]:
#code to make submission CSV file
submit_tensor=None
for i, vid in enumerate(sorted(os.listdir(TEST_FEATS))):
    video_feat = TEST_FEATS + vid
    top5, _ = model.caption_image(torch.tensor(np.load(video_feat)))
    top5 = top5.int()
    for ix, v in enumerate(top5[1, :]):
        if vocab_itos[v.item()] in relationship_vocab_stoi.keys():
            top5[1, ix] = relationship_vocab_stoi[vocab_itos[v.item()]]
        else:
            top5[1, ix] = obj1_obj2[vocab_itos[v.item()]]
    if i==0: 
        submit_tensor = top5
    else:
        submit_tensor = torch.cat((submit_tensor, top5), dim=0)

px = submit_tensor.numpy()
px = pd.DataFrame(px)
px['label'] = px.iloc[:, 0:5].apply(lambda x: " ".join(x.astype(str)), axis=1)
px.drop([0, 1, 2, 3, 4], axis = 1, inplace=True)
# px.index = np.arange(1, len(px) + 1)
px.reset_index(inplace=True)
px = px.rename(columns = {'index':'Id'})
px.to_csv('/content/drive/MyDrive/cs5242_project/submissions/best_submission.csv', index=False, header=True)